This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [4]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')


Python-dotenv could not parse statement starting at line 2


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [5]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [6]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?limit=1&api_key='+API_KEY

In [7]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r= requests.get(url)
print(r.json())
afx= r.json()

{'dataset_data': {'limit': 1, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2000-06-07', 'end_date': '2020-12-01', 'frequency': 'daily', 'data': [['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:




1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [8]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2018-01-01&api_key='+API_KEY
query = requests.get(url)


2. Convert the returned JSON object into a Python dictionary.

In [9]:
AFX= query.json()

In [10]:
print(type(AFX))

<class 'dict'>


In [11]:
import json
exporable_json= json.dumps(AFX, indent=4)

#Initially i was printing, but for submission to Git I will just store this variable


3. Calculate what the highest and lowest opening prices were for the stock in this period.


In [12]:
open_dict = []

for data in AFX['dataset_data']['data']: 
    if type(data[1])== float:
        open_dict.append(data[1])

maximum = max(open_dict)
minimum = min(open_dict)
print('Highest price: $' + str(maximum))
print('Lowest price: $' + str(minimum))




Highest price: $53.11
Lowest price: $34.0


4. What was the largest change in any one day (based on High and Low price)?

In [13]:
delta= []

for data in AFX['dataset_data']['data']:
    if type(data[2])== float and type(data[3]==float):
        delta.append(data[2]-data[3])
        
maximum_delta= max(delta)
print ('The maximum delta is: ' + str(maximum_delta))

The maximum delta is: 2.8100000000000023


5. What was the largest change between any two days (based on Closing Price)?


In [14]:
column_names = AFX['dataset_data']['column_names']
data = AFX['dataset_data']['data']

close_idx = column_names.index('Close')

max_change = 0.0
prev_day_close = data[0][close_idx] if data else None


for day in data[1:]:  
    current_day_close = day[close_idx]
    if current_day_close is not None and prev_day_close is not None:
        change = abs(current_day_close - prev_day_close)
        max_change = max(max_change, change)  
    prev_day_close = current_day_close  

print("The largest change between any two consecutive days based on Closing Price was approximately $", max_change)

The largest change between any two consecutive days based on Closing Price was approximately $ 2.559999999999995


6. What was the average daily trading volume during this year?


In [15]:
volumes = []

for data in AFX['dataset_data']['data']: 
    if type(data[6]) is not None:
        volumes.append(data[6])
        
avg = sum(volumes) / len(volumes)
print('The average daily trading volume in 2017 was:  ' + str(avg))

The average daily trading volume in 2017 was:  89124.33725490196


7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [16]:
volumes = []

for data in AFX['dataset_data']['data']: 
    if type(data[6]) is not None:
        volumes.append(data[6])
        
volumes.sort()

# Calculate the median
if len(volumes) % 2 == 0:  # If there is an even number of elements, the median is the average of the two middle numbers
    median = (volumes[len(volumes) // 2 - 1] + volumes[len(volumes) // 2]) / 2
else:  # If there is an odd number of elements, the median is the middle element
    median = volumes[len(volumes) // 2]

# Output the median trading volume
print('The median daily trading volume in 2017 was: ' + str(median))

The median daily trading volume in 2017 was: 76286.0
